# Jupyter Notebooklets Demo

### [@ianhellen](https://twitter.com/ianhellen)
#### Principal Dev - MSTIC, Azure Security

# What are notebooklets?

Collections of notebook cells that implement some useful reusable sequence

## Rationale
- Notebook code can quickly become complex and length:
  - Can obscure the information you are trying to display
  - Can be intimidating to non-developers
- Notebook code cells are not easily re-useable:
  - You can copy and paste but how do you sync changes back to original notebook?
  - Difficult to discover code snippets in notebooks
- Notebook code is often fragile:
  - Often not parameterized
  - Code blocks are frequently dependent on global values assigned earlier
  - Output data is not in any standard format
  - Difficult to test

## Characteristics
- One or small number of entry points
- Must be paramertizable
- Can query, process or visualize data (or any combination)
- Typically return a result or package of results for use later in the notebook


In [1]:
import sys
from IPython.display import display, HTML, Markdown

from msticpy.nbtools.nbinit import init_notebook
init_notebook(namespace=globals(), extra_imports=["ipwhois, IPWhois"])
import os
os.environ["KQLMAGIC_LOAD_MODE"] = "silent"

Processing imports....
Checking configuration....
Setting options....


---
# Notebooklets in use

## Import the package

### Calling init() 
- Discovers and imports notebooklet classes/modules
- Loads required data providers
- Authenticates to providers if required at startup
- Can supply list of providers to load
- Can pass parameters to each provider (settings loaded from config by default)

In [2]:
# pip install git+https://github.com/microsoft/msticnb

In [3]:
import msticnb as nb
nb.init()

3 notebooklets loaded.
Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


Loaded providers: azure_sentinel, ti_lookup, geolite_lookup


---
## Notebooklet classes are discovered and imported 
#### `nblts` attribute exposes notebooklets (niblets?) in a tree structure
- Useful for autocomplete when you more or less know what you're looking for

In [4]:
nb.nblts

azsent

### Also exposed in `nb_index` keyed by relative path

In [5]:
nb.nb_index

{'nblts.azsent.host.HostSummary': msticnb.nb.azsent.host.host_summary.HostSummary,
 'nblts.azsent.host.WinHostEvents': msticnb.nb.azsent.host.win_host_events.WinHostEvents,
 'nblts.azsent.network.NetworkFlowSummary': msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary}

## Also a find function that looks for:
- words or regex
- searches class docstring
- metadata such as entities supported and options supported

In [6]:
nb.find("host, net.*", full_match=True)

[('HostSummary', msticnb.nb.azsent.host.host_summary.HostSummary),
 ('NetworkFlowSummary',
  msticnb.nb.azsent.network.network_flow_summary.NetworkFlowSummary)]

---
# More detailed (and user-friendly) help in the `show_help()` method

In [7]:
nb.nblts.azsent.host.HostSummary.show_help()

---
# How are they used?

## Most require time range parameters

In [8]:
time_span = nbwidgets.QueryTime(auto_display=True, units="day", origin_time=pd.to_datetime("2019-02-10"), before=10)
from msticnb.common import TimeSpan

HTML(value='<h4>Set query time boundaries</h4>')

## Run the notebooklet using the `run()` method

>  **Note:** You'll want to assign the return value of `run()` to something or terminate with a semicolon<br>
>  Both the notebooklet and the return `result` class generate displayable output - so you'll get
>  a lot of duplicated output.

In [9]:
host_summary = nb.nblts.azsent.host.HostSummary()
host_sum_rslt = host_summary.run(value="Msticalertswin1", timespan=TimeSpan(time_selector=time_span))

Getting data from SecurityEvent...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Getting data from Syslog...


<IPython.core.display.Javascript object>

...

Loading BokehJS ...

Getting data from Bookmarks...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

No bookmarks found.


## Result classes content can be displayed in the notebook
Use `display(result)` if you want to display the content in the middle of a cell

In [10]:
host_sum_rslt

HostSummaryResult(description='Host summary', _attribute_desc={'host_entity': ('msticpy.data.nbtools.entities.Host', 'The host entity object contains data about the host such as name, environment, operating system version, IP addresses and Azure VM details. Depending on the type of host, not all of this data may be populated.'), 'related_alerts': ('pd.DataFrame', 'Pandas DataFrame of any alerts recorded for the host within the query time span.'), 'alert_timeline': ('', 'Bokeh time plot of alerts recorded for host.'), 'related_bookmarks': ('pd.DataFrame', 'Pandas DataFrame of any investigation bookmarks relating to the host.')}, host_entity=Host(Type=host, HostName=MSTICAlertsWin1, SourceComputerId=263a788b-6526-4cdc-8ed9-d7...), related_alerts=                                 TenantId       TimeGenerated  \
0    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-15 20:27:38   
1    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 18:03:23   
2    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 11:52:06   
3    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 11:52:06   
4    52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-14 05:19:13   
..                                    ...                 ...   
313  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-05 17:15:18   
314  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 07:28:11   
315  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 07:58:31   
316  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 06:58:31   
317  52b1ab41-869e-4138-9e40-2a4457f09bf0 2019-02-06 08:58:32   

                                   AlertDisplayName  \
0                      Suspicious Activity Detected   
1                        Security incident detected   
2                Suspicious system process executed   
3    Potential attempt to bypass AppLocker detected   
4    Security incident with shared process detected   
..                                              ...   
313             Suspicious SVCHOST process executed   
314  Security incident with shared process detected   
315                               Sample Alert Rule   
316                               Sample Alert Rule   
317                               Sample Alert Rule   

                                          AlertName Severity  \
0                      Suspicious Activity Detected   Medium   
1                        Security incident detected     High   
2                Suspicious system process executed   Medium   
3    Potential attempt to bypass AppLocker detected     High   
4    Security incident with shared process detected     High   
..                                              ...      ...   
313             Suspicious SVCHOST process executed     High   
314  Security incident with shared process detected     High   
315                               Sample Alert Rule   Medium   
316                               Sample Alert Rule   Medium   
317                               Sample Alert Rule   Medium   

                                                                                             Description  \
0    Analysis of host data has detected a sequence of one or more processes running on MSTICAlertsWin...   
1    The incident which started on 2019-02-14 11:51:38 UTC and recently detected on 2019-02-14 18:03:...   
2    The system process c:\windows\fonts\conhost.exe was observed running in an abnormal context. Mal...   
3    Analysis of host data on MSTICALERTSWIN1 detected a potential attempt to bypass AppLocker restri...   
4    The incident which started on 2019-02-12 11:48:01 UTC and recently detected on 2019-02-14 05:19:...   
..                                                                                                   ...   
313  The system process SVCHOST was observed running in an abnormal context. Malware often use SVCHOS...   
314  The incident which started on 2019-02-05 21:15:07 UTC and recently detected on 2019-02-06 07:28:...   
315                                                            

---
## Simple Notebooklet browser

In [11]:
nb.browse()

In [12]:
# value="MSTICAlertsWin1", timespan=time_span

win_host_events = nb.nblts.azsent.host.WinHostEvents()
timespan = TimeSpan(start="2020-05-07 00:10")
win_host_events_rslt = win_host_events.run(value="MSTICAlertsWin1", timespan=time_span)

Getting data from SecurityEvent...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Activity,.NET v4.5,.NET v4.5 Classic,DWM-1,DWM-2,DWM-3,DWM-4,IUSR,LOCAL SERVICE,MSTICAdmin,MSTICAlertsWin1$,NETWORK SERVICE,No Account,SYSTEM,adm1nistrator,ian
1100 - The event logging service has shut down.,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
4608 - Windows is starting up.,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
4616 - The system time was changed.,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0
4625 - An account failed to log on.,0,0,0,0,0,0,0,0,1,0,0,0,0,3,5
4634 - An account was logged off.,1,1,0,10,0,2,0,0,75,0,0,0,0,8,16
4647 - User initiated logoff.,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
4648 - A logon was attempted using explicit credentials.,0,0,0,0,0,0,0,0,8,90,0,0,0,0,0
4672 - Special privileges assigned to new logon.,1,1,6,12,2,2,3,3,78,0,3,0,442,0,14
4717 - System security access was granted to an account.,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0
4720 - A user account was created.,0,0,0,0,0,0,0,0,19,0,0,0,0,0,2


Activity,MSTICAdmin,MSTICAlertsWin1$,ian
4720 - A user account was created.,19,0,2
4722 - A user account was enabled.,19,0,2
4724 - An attempt was made to reset an account's password.,36,0,4
4726 - A user account was deleted.,19,0,2
4728 - A member was added to a security-enabled global group.,19,0,2
4729 - A member was removed from a security-enabled global group.,19,0,2
4732 - A member was added to a security-enabled local group.,27,0,4
4733 - A member was removed from a security-enabled local group.,26,1,4
4738 - A user account was changed.,46,0,6


Loading BokehJS ...

## Additional operations apart from `run()`
We can use expand events to unpack the `EventData` column for selected EventIDs

In [13]:
win_host_events_rslt.account_events.head(5)

,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,...,TargetUserName,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId
1064,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 13:46:04.077,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,,,,,,,,,,,,,...,None,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,a2419b5a-5718-4ca9-9daa-594a48380632,00000000-0000-0000-0000-000000000001,2019-02-18 13:46:04.990,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
1065,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 13:46:04.077,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4720,4720 - A user account was created.,,,,,,,,,,%%1794,,,...,qf938$,,,,,,,,,,\t\t%%2080 \t\t%%2082 \t\t%%2084,%%1793,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,3bb2d72c-d487-4e52-a367-a678a0a8110a,00000000-0000-0000-0000-000000000001,2019-02-18 13:46:04.990,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
1066,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 13:46:04.077,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4722,4722 - A user account was enabled.,,,,,,,,,,,,,...,qf938$,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,bfbcc3c2-0a15-4a4a-8b50-4b83a43cd101,00000000-0000-0000-0000-000000000001,2019-02-18 13:46:04.990,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
1067,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 13:46:04.077,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Dummy...",4738,4738 - A user account was changed.,,,,,,,,,,%%1794,,,...,qf938$,,,,,,,,,,\t\t%%2048 \t\t%%2050,-,-,%%1793,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27de5de7-a634-467d-b0f8-8a61d389af5b,00000000-0000-0000-0000-000000000001,2019-02-18 13:46:04.990,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
1068,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 13:46:04.077,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13824,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Targe...",4724,4724 - An attempt was made to reset an account's password.,,,,,,,,,,,,,...,qf938$,,,,,,,,,,,,,,,,,,263a788b-6526-4cdc-8ed9-d79402fe4aa0,38695561-062d-4a27-b8c3-3ba665e4c5fe,00000000-0000-0000-0000-000000000001,2019-02-18 13:46:04.990,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/reso

In [14]:
win_host_events.expand_events(event_ids=4728).head(5)

Parsing event data...


,TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,MemberName,MemberSid,PrivilegeList,SubjectAccount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TargetAccount,TargetDomainName,TargetSid,TargetUserName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,Type,_ResourceId
1064,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-18 13:46:04.077,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1122,-,MSTICAlertsWin1\ian,MSTICAlertsWin1,0x52884d4,ian,S-1-5-21-996632719-2361334927-4038480536-1120,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,a2419b5a-5718-4ca9-9daa-594a48380632,00000000-0000-0000-0000-000000000001,2019-02-18 13:46:04.990,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
2196,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.173,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1118,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,27df6071-1e81-4e24-934c-dc96667b83ab,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
2207,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-11 09:58:50.447,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1119,-,MSTICAlertsWin1\MSTICAdmin,MSTICAlertsWin1,0xbd57571,MSTICAdmin,S-1-5-21-996632719-2361334927-4038480536-500,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,73b0fe4e-9886-43ab-afa6-b43eb7434402,00000000-0000-0000-0000-000000000001,2019-02-11 09:58:51.400,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
3195,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-15 20:14:36.130,OpsManager,MSTICAlertsWin1\ian,User,MSTICAlertsWin1,Microsoft-Windows-Security-Auditing,Security,13826,8,"<EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event"">\r\n <Data Name=""Membe...",4728,4728 - A member was added to a security-enabled global group.,-,S-1-5-21-996632719-2361334927-4038480536-1121,-,MSTICAlertsWin1\ian,MSTICAlertsWin1,0x1e8ae56,ian,S-1-5-21-996632719-2361334927-4038480536-1120,MSTICAlertsWin1\None,MSTICAlertsWin1,S-1-5-21-996632719-2361334927-4038480536-513,None,263a788b-6526-4cdc-8ed9-d79402fe4aa0,a546863d-0dc5-4fd5-8fe0-fb54ee61fa5a,00000000-0000-0000-0000-000000000001,2019-02-15 20:14:37.483,AOI-52b1ab41-869e-4138-9e40-2a4457f09bf0,SecurityEvent,/subscriptions/40dcc8bf-0478-4f3b-b275-ed0a94f2c013/resourcegroups/asihuntomsworkspacerg/provide...
3879,52b1ab41-869e-4138-9e40-2a4457f09bf0,2019-02-08 09:27:35.953,OpsManager,MSTICAlertsWin1\MSTICAdmin,User,MSTICAle

---
# Anatomy of a Notebooklet

# Three sections:
- Results class - what is it going to return
- Notebooklet class - `run()` defines what the notebooklet does
- Code - series of functions that do the actual work

In [15]:
nb.nblts.azsent.host.WinHostEvents.import_cell()

In [ ]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------
"""Notebooklet for Windows Security Events."""
import pkgutil
import os
from typing import Any, Optional, Iterable, Union
from defusedxml import ElementTree
from defusedxml.ElementTree import ParseError

import attr
from bokeh.plotting.figure import Figure
from bokeh.models import LayoutDOM
from IPython.display import display
import numpy as np
import pandas as pd
from msticpy.common.utility import md
from msticpy.nbtools import nbdisplay

from msticnb.common import (
    TimeSpan,
    MsticnbMissingParameterError,
    print_data_wait,
    print_status,
    set_text,
)
from msticnb.notebooklet import Notebooklet, NotebookletResult, NBMetaData

from msticnb._version import VERSION

__version__ = VERSION
__author__ = "Ian Hellen"


@attr.s(auto_attribs=True)
class WinHostEventsResult(NotebookletResult):
    """
    Windows Host Security Events Results.

    Attributes
    ----------
    all_events : pd.DataFrame
        DataFrame of all raw events retrieved.
    event_pivot : pd.DataFrame
        DataFrame that is a pivot table of event ID
        vs. Account
    account_events : pd.DataFrame
        DataFrame containing a subset of account management
        events such as account and group modification.
    acct_pivot : pd.DataFrame
        DataFrame that is a pivot table of event ID
        vs. Account of account management events
    account_timeline : Union[Figure, LayoutDOM]
        Bokeh plot figure or Layout showing the account events on an
        interactive timeline.
    expanded_events : pd.DataFrame
        If `expand_events` option is specified, this will contain
        the parsed/expanded EventData as individual columns.

    """

    description: str = "Windows Host Security Events"
    all_events: pd.DataFrame = None
    event_pivot: pd.DataFrame = None
    account_events: pd.DataFrame = None
    account_pivot: pd.DataFrame = None
    account_timeline: Union[Figure, LayoutDOM] = None
    expanded_events: pd.DataFrame = None


class WinHostEvents(Notebooklet):
    """
    Windows host Security Events Notebooklet class.

    Queries and displays Windows Security Events including:

    - All security events summary
    - Extracting and displaying account management events
    - Account management event timeline
    - Optionally parsing packed event data into DataFrame columns

    Process (4688) and Account Logon (4624, 4625) are not included
    in the event types processed by this module.

    Default Options
    ---------------
    - event_pivot: Display a summary of all event types.
    - acct_events: Display a summary and timeline of account
      management events.

    Other Options
    -------------
    - expand_events: parses the XML EventData column into separate
      DataFrame columns. This can be very expensive with a large
      event set. We recommend using the expand_events() method to
      select a specific subset of events to process.

    """

    metadata = NBMetaData(
        name=__qualname__,  # type: ignore  # noqa
        mod_name=__name__,
        description="Window security events summary",
        default_options=["event_pivot", "acct_events"],
        other_options=["expand_events"],
        keywords=["host", "computer", "events", "windows", "account"],
        entity_types=["host"],
        req_providers=["azure_sentinel"],
    )

    @set_text(
        title="Host Security Events Summary",
        hd_level=1,
        text="Data and plots are store in the result class returned by this function",
    )
    def run(
        self,
        value: Any = None,
        data: Optional[pd.DataFrame] = None,
        timespan: Optional[TimeSpan] = None,
        options: Optional[Iterable[str]] = None,
        **kwargs,
    ) -> WinHostEventsResult:
        """
        Return Windows Security Event summary.

        Parameters
        ----------
        value : str
            Host name
        data : Optional[pd.DataFrame], optional
            Not used, by default None
        timespan : TimeSpan
            Timespan over which operations such as queries will be
            performed, by default None.
            This can be a TimeStamp object or another object that
            has valid `start`, `end`, or `period` attributes.
        options : Optional[Iterable[str]], optional
            List of options to use, by default None.
            A value of None means use default options.
            Options prefixed with "+" will be added to the default options.
            To see the list of available options type `help(cls)` where
            "cls" is the notebooklet class or an instance of this class.

        Other Parameters
        ----------------
        start : Union[datetime, datelike-string]
            Alternative to specifying timespan parameter.
        end : Union[datetime, datelike-string]
            Alternative to specifying timespan parameter.

        Returns
        -------
        HostSummaryResult
            Result object with attributes for each result type.

        Raises
        ------
        MsticnbMissingParameterError
            If required parameters are missing

        """
        super().run(
            value=value, data=data, timespan=timespan, options=options, **kwargs
        )

        if not value:
            raise MsticnbMissingParameterError("value")
        if not timespan:
            raise MsticnbMissingParameterError("timespan.")

        result = WinHostEventsResult()

        all_events_df, event_pivot_df = _get_win_security_events(
            self.query_provider, host_name=value, timespan=self.timespan
        )
        result.all_events = all_events_df
        result.event_pivot = event_pivot_df

        if "event_pivot" in self.options:
            _display_event_pivot(event_pivot=event_pivot_df)

        if "acct_events" in self.options:
            result.account_events = _extract_acct_mgmt_events(event_data=all_events_df)
            result.account_pivot = _create_acct_event_pivot(
                account_event_data=result.account_events
            )
            _display_acct_event_pivot(event_pivot_df=result.account_pivot)
            result.account_timeline = _display_acct_mgmt_timeline(
                acct_event_data=result.account_events
            )

        if "expand_events" in self.options:
            result.expanded_events = _parse_eventdata(all_events_df)

        md("To unpack eventdata from selected events use expand_events()")
        self._last_result = result  # pylint: disable=attribute-defined-outside-init
        return self._last_result

    def expand_events(
        self, event_ids: Optional[Union[int, Iterable[int]]] = None
    ) -> pd.DataFrame:
        """
        Expand `EventData` for `event_ids` into separate columns.

        Parameters
        ----------
        event_ids : Optional[Union[int, Iterable[int]]], optional
            Single or interable of event IDs (ints).
            If no event_ids are specified all events will be expanded.

        Returns
        -------
        pd.DataFrame
            Results with expanded columns.

        Notes
        -----
        For a specific event ID you can expand the EventProperties values
        into their own columns using this function.
        You can do this for the whole data set but it will time-consuming
        and result in a lot of sparse columns in the output data frame.

        """
        if (
            not self._last_result or self._last_result.all_events is None
        ):  # type: ignore
            print(
                "Please use 'run()' to fetch the data before using this method.",
                "\nThen call 'expand_events()'",
            )
            return None
        return _parse_eventdata(
            event_data=self._last_result.all_events,  # type: ignore
            event_ids=event_ids,
        )


# %%
# Get Windows Security Events
def _get_win_security_events(qry_prov, host_name, timespan):
    print_data_wait("SecurityEvent")

    all_events_df = qry_prov.WindowsSecurity.list_host_events(
        timespan,
        host_name=host_name,
        add_query_items="| where EventID != 4688 and EventID != 4624",
    )

    # Create a pivot of Event vs. Account
    win_events_acc = all_events_df[["Account", "Activity", "TimeGenerated"]].copy()
    win_events_acc = win_events_acc.replace("-\\-", "No Account").replace(
        {"Account": ""}, value="No Account"
    )
    win_events_acc["Account"] = win_events_acc.apply(
        lambda x: x.Account.split("\\")[-1], axis=1
    )
    event_pivot_df = (
        pd.pivot_table(
            win_events_acc,
            values="TimeGenerated",
            index=["Activity"],
            columns=["Account"],
            aggfunc="count",
        )
        .fillna(0)
        .reset_index()
    )
    return all_events_df, event_pivot_df


@set_text(
    title="Summary of Security Events on host",
    text="""
Yellow highlights indicate account with highest event count.
""",
)
def _display_event_pivot(event_pivot):
    display(
        event_pivot.style.applymap(lambda x: "color: white" if x == 0 else "")
        .applymap(
            lambda x: "background-color: lightblue"
            if not isinstance(x, str) and x > 0
            else ""
        )
        .set_properties(subset=["Activity"], **{"width": "400px", "text-align": "left"})
        .highlight_max(axis=1)
        .hide_index()
    )


# %%
# Extract event details from events
SCHEMA = "http://schemas.microsoft.com/win/2004/08/events/event"


def _parse_event_data_row(row):
    try:
        xdoc = ElementTree.fromstring(row.EventData)
        col_dict = {
            elem.attrib["Name"]: elem.text for elem in xdoc.findall(f"{{{SCHEMA}}}Data")
        }
        reassigned = set()
        for key, val in col_dict.items():
            if key in row and not row[key]:
                row[key] = val
                reassigned.add(key)
        if reassigned:
            for key in reassigned:
                col_dict.pop(key)
        return col_dict
    except (ParseError, TypeError):
        return None


def _expand_event_properties(input_df):
    # For a specific event ID you can explode the EventProperties values
    # into their own columns using this function. You can do this for
    # the whole data set but it will result
    # in a lot of sparse columns in the output data frame.
    exp_df = input_df.apply(lambda x: pd.Series(x.EventProperties), axis=1)
    return (
        exp_df.drop(set(input_df.columns).intersection(exp_df.columns), axis=1)
        .merge(
            input_df.drop("EventProperties", axis=1),
            how="inner",
            left_index=True,
            right_index=True,
        )
        .replace("", np.nan)  # these 3 lines get rid of blank columns
        .dropna(axis=1, how="all")
        .fillna("")
    )


@set_text(
    title="Parsing eventdata into columns",
    hd_level=3,
    text="""
This may take some time to complete for large numbers of events.

Since event types have different schema, some of the columns will
not be populated for certain Event IDs and will show as `NaN`.
""",
    md=True,
)
def _parse_eventdata(event_data, event_ids: Optional[Union[int, Iterable[int]]] = None):
    if event_ids:
        if isinstance(event_ids, int):
            event_ids = [event_ids]
        src_event_data = event_data[event_data["EventID"].isin(event_ids)].copy()
    else:
        src_event_data = event_data.copy()

    # Parse event properties into a dictionary
    print_status("Parsing event datamsticnb.")
    src_event_data["EventProperties"] = src_event_data.apply(
        _parse_event_data_row, axis=1
    )
    return _expand_event_properties(src_event_data)


# %%
# Account management events
def _extract_acct_mgmt_events(event_data):
    # Get a full list of Windows Security Events

    w_evt = pkgutil.get_data("msticpy", f"resources{os.sep}WinSecurityEvent.json")
    win_event_df = pd.read_json(w_evt)

    # Create criteria for events that we're interested in
    acct_sel = win_event_df["subcategory"] == "User Account Management"
    group_sel = win_event_df["subcategory"] == "Security Group Management"
    schtask_sel = (win_event_df["subcategory"] == "Other Object Access Events") & (
        win_event_df["description"].str.contains("scheduled task")
    )

    event_list = win_event_df[acct_sel | group_sel | schtask_sel]["event_id"].to_list()
    # Add Service install event
    event_list.append(7045)
    return event_data[event_data["EventID"].isin(event_list)]


def _create_acct_event_pivot(account_event_data):
    # Create a pivot of Event vs. Account
    win_events_acc = account_event_data[["Account", "Activity", "TimeGenerated"]].copy()
    win_events_acc = win_events_acc.replace("-\\-", "No Account").replace(
        {"Account": ""}, value="No Account"
    )
    win_events_acc["Account"] = win_events_acc.apply(
        lambda x: x.Account.split("\\")[-1], axis=1
    )
    event_pivot_df = (
        pd.pivot_table(
            win_events_acc,
            values="TimeGenerated",
            index=["Activity"],
            columns=["Account"],
            aggfunc="count",
        )
        .fillna(0)
        .reset_index()
    )
    return event_pivot_df


@set_text(
    title="Summary of Account Management Events on host",
    text="""
Yellow highlights indicate account with highest event count.
""",
)
def _display_acct_event_pivot(event_pivot_df):
    display(
        event_pivot_df.style.applymap(lambda x: "color: white" if x == 0 else "")
        .applymap(
            lambda x: "background-color: lightblue"
            if not isinstance(x, str) and x > 0
            else ""
        )
        .set_properties(subset=["Activity"], **{"width": "400px", "text-align": "left"})
        .highlight_max(axis=1)
        .hide_index()
    )


@set_text(title="Timeline of Account Management Events on host")
def _display_acct_mgmt_timeline(acct_event_data):
    # Plot events on a timeline
    return nbdisplay.display_timeline(
        data=acct_event_data,
        group_by="EventID",
        source_columns=["Activity", "Account"],
        legend="right",
    )


In [16]:
# -------------------------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License. See License.txt in the project root for
# license information.
# --------------------------------------------------------------------------
"""Notebooklet for Windows Security Events."""
import pkgutil
import os
from typing import Any, Optional, Iterable, Union
from defusedxml import ElementTree
from defusedxml.ElementTree import ParseError

import attr
from bokeh.plotting.figure import Figure
from bokeh.models import LayoutDOM
from IPython.display import display
import numpy as np
import pandas as pd
from msticpy.common.utility import md
from msticpy.nbtools import nbdisplay

from msticnb.common import (
    TimeSpan,
    MsticnbMissingParameterError,
    print_data_wait,
    print_status,
    set_text,
)
from msticnb.notebooklet import Notebooklet, NotebookletResult, NBMetaData

from msticnb._version import VERSION

__version__ = VERSION
__author__ = "Ian Hellen"


@attr.s(auto_attribs=True)
class WinHostEventsResult(NotebookletResult):
    """
    Windows Host Security Events Results.

    Attributes
    ----------
    all_events : pd.DataFrame
        DataFrame of all raw events retrieved.
    event_pivot : pd.DataFrame
        DataFrame that is a pivot table of event ID
        vs. Account
    account_events : pd.DataFrame
        DataFrame containing a subset of account management
        events such as account and group modification.
    acct_pivot : pd.DataFrame
        DataFrame that is a pivot table of event ID
        vs. Account of account management events
    account_timeline : Union[Figure, LayoutDOM]
        Bokeh plot figure or Layout showing the account events on an
        interactive timeline.
    expanded_events : pd.DataFrame
        If `expand_events` option is specified, this will contain
        the parsed/expanded EventData as individual columns.

    """

    description: str = "Windows Host Security Events"
    all_events: pd.DataFrame = None
    event_pivot: pd.DataFrame = None
    account_events: pd.DataFrame = None
    account_pivot: pd.DataFrame = None
    account_timeline: Union[Figure, LayoutDOM] = None
    expanded_events: pd.DataFrame = None


class WinHostEvents(Notebooklet):
    """
    Windows host Security Events Notebooklet class.

    Queries and displays Windows Security Events including:

    - All security events summary
    - Extracting and displaying account management events
    - Account management event timeline
    - Optionally parsing packed event data into DataFrame columns

    Process (4688) and Account Logon (4624, 4625) are not included
    in the event types processed by this module.

    Default Options
    ---------------
    - event_pivot: Display a summary of all event types.
    - acct_events: Display a summary and timeline of account
      management events.

    Other Options
    -------------
    - expand_events: parses the XML EventData column into separate
      DataFrame columns. This can be very expensive with a large
      event set. We recommend using the expand_events() method to
      select a specific subset of events to process.

    """

    metadata = NBMetaData(
        name=__qualname__,  # type: ignore  # noqa
        mod_name=__name__,
        description="Window security events summary",
        default_options=["event_pivot", "acct_events"],
        other_options=["expand_events"],
        keywords=["host", "computer", "events", "windows", "account"],
        entity_types=["host"],
        req_providers=["azure_sentinel"],
    )

    @set_text(
        title="Host Security Events Summary",
        hd_level=1,
        text="Data and plots are store in the result class returned by this function",
    )
    def run(
        self,
        value: Any = None,
        data: Optional[pd.DataFrame] = None,
        timespan: Optional[TimeSpan] = None,
        options: Optional[Iterable[str]] = None,
        **kwargs,
    ) -> WinHostEventsResult:
        """
        Return Windows Security Event summary.

        Parameters
        ----------
        value : str
            Host name
        data : Optional[pd.DataFrame], optional
            Not used, by default None
        timespan : TimeSpan
            Timespan over which operations such as queries will be
            performed, by default None.
            This can be a TimeStamp object or another object that
            has valid `start`, `end`, or `period` attributes.
        options : Optional[Iterable[str]], optional
            List of options to use, by default None.
            A value of None means use default options.
            Options prefixed with "+" will be added to the default options.
            To see the list of available options type `help(cls)` where
            "cls" is the notebooklet class or an instance of this class.

        Other Parameters
        ----------------
        start : Union[datetime, datelike-string]
            Alternative to specifying timespan parameter.
        end : Union[datetime, datelike-string]
            Alternative to specifying timespan parameter.

        Returns
        -------
        HostSummaryResult
            Result object with attributes for each result type.

        Raises
        ------
        MsticnbMissingParameterError
            If required parameters are missing

        """
        super().run(
            value=value, data=data, timespan=timespan, options=options, **kwargs
        )

        if not value:
            raise MsticnbMissingParameterError("value")
        if not timespan:
            raise MsticnbMissingParameterError("timespan.")

        result = WinHostEventsResult()

        all_events_df, event_pivot_df = _get_win_security_events(
            self.query_provider, host_name=value, timespan=self.timespan
        )
        result.all_events = all_events_df
        result.event_pivot = event_pivot_df

        if "event_pivot" in self.options:
            _display_event_pivot(event_pivot=event_pivot_df)

        if "acct_events" in self.options:
            result.account_events = _extract_acct_mgmt_events(event_data=all_events_df)
            result.account_pivot = _create_acct_event_pivot(
                account_event_data=result.account_events
            )
            _display_acct_event_pivot(event_pivot_df=result.account_pivot)
            result.account_timeline = _display_acct_mgmt_timeline(
                acct_event_data=result.account_events
            )

        if "expand_events" in self.options:
            result.expanded_events = _parse_eventdata(all_events_df)

        md("To unpack eventdata from selected events use expand_events()")
        self._last_result = result  # pylint: disable=attribute-defined-outside-init
        return self._last_result

    def expand_events(
        self, event_ids: Optional[Union[int, Iterable[int]]] = None
    ) -> pd.DataFrame:
        """
        Expand `EventData` for `event_ids` into separate columns.

        Parameters
        ----------
        event_ids : Optional[Union[int, Iterable[int]]], optional
            Single or interable of event IDs (ints).
            If no event_ids are specified all events will be expanded.

        Returns
        -------
        pd.DataFrame
            Results with expanded columns.

        Notes
        -----
        For a specific event ID you can expand the EventProperties values
        into their own columns using this function.
        You can do this for the whole data set but it will time-consuming
        and result in a lot of sparse columns in the output data frame.

        """
        if (
            not self._last_result or self._last_result.all_events is None
        ):  # type: ignore
            print(
                "Please use 'run()' to fetch the data before using this method.",
                "\nThen call 'expand_events()'",
            )
            return None
        return _parse_eventdata(
            event_data=self._last_result.all_events,  # type: ignore
            event_ids=event_ids,
        )


# %%
# Get Windows Security Events
def _get_win_security_events(qry_prov, host_name, timespan):
    print_data_wait("SecurityEvent")

    all_events_df = qry_prov.WindowsSecurity.list_host_events(
        timespan,
        host_name=host_name,
        add_query_items="| where EventID != 4688 and EventID != 4624",
    )

    # Create a pivot of Event vs. Account
    win_events_acc = all_events_df[["Account", "Activity", "TimeGenerated"]].copy()
    win_events_acc = win_events_acc.replace("-\\-", "No Account").replace(
        {"Account": ""}, value="No Account"
    )
    win_events_acc["Account"] = win_events_acc.apply(
        lambda x: x.Account.split("\\")[-1], axis=1
    )
    event_pivot_df = (
        pd.pivot_table(
            win_events_acc,
            values="TimeGenerated",
            index=["Activity"],
            columns=["Account"],
            aggfunc="count",
        )
        .fillna(0)
        .reset_index()
    )
    return all_events_df, event_pivot_df


@set_text(
    title="Summary of Security Events on host",
    text="""
Yellow highlights indicate account with highest event count.
""",
)
def _display_event_pivot(event_pivot):
    display(
        event_pivot.style.applymap(lambda x: "color: white" if x == 0 else "")
        .applymap(
            lambda x: "background-color: lightblue"
            if not isinstance(x, str) and x > 0
            else ""
        )
        .set_properties(subset=["Activity"], **{"width": "400px", "text-align": "left"})
        .highlight_max(axis=1)
        .hide_index()
    )


# %%
# Extract event details from events
SCHEMA = "http://schemas.microsoft.com/win/2004/08/events/event"


def _parse_event_data_row(row):
    try:
        xdoc = ElementTree.fromstring(row.EventData)
        col_dict = {
            elem.attrib["Name"]: elem.text for elem in xdoc.findall(f"{{{SCHEMA}}}Data")
        }
        reassigned = set()
        for key, val in col_dict.items():
            if key in row and not row[key]:
                row[key] = val
                reassigned.add(key)
        if reassigned:
            for key in reassigned:
                col_dict.pop(key)
        return col_dict
    except (ParseError, TypeError):
        return None


def _expand_event_properties(input_df):
    # For a specific event ID you can explode the EventProperties values
    # into their own columns using this function. You can do this for
    # the whole data set but it will result
    # in a lot of sparse columns in the output data frame.
    exp_df = input_df.apply(lambda x: pd.Series(x.EventProperties), axis=1)
    return (
        exp_df.drop(set(input_df.columns).intersection(exp_df.columns), axis=1)
        .merge(
            input_df.drop("EventProperties", axis=1),
            how="inner",
            left_index=True,
            right_index=True,
        )
        .replace("", np.nan)  # these 3 lines get rid of blank columns
        .dropna(axis=1, how="all")
        .fillna("")
    )


@set_text(
    title="Parsing eventdata into columns",
    hd_level=3,
    text="""
This may take some time to complete for large numbers of events.

Since event types have different schema, some of the columns will
not be populated for certain Event IDs and will show as `NaN`.
""",
    md=True,
)
def _parse_eventdata(event_data, event_ids: Optional[Union[int, Iterable[int]]] = None):
    if event_ids:
        if isinstance(event_ids, int):
            event_ids = [event_ids]
        src_event_data = event_data[event_data["EventID"].isin(event_ids)].copy()
    else:
        src_event_data = event_data.copy()

    # Parse event properties into a dictionary
    print_status("Parsing event datamsticnb.")
    src_event_data["EventProperties"] = src_event_data.apply(
        _parse_event_data_row, axis=1
    )
    return _expand_event_properties(src_event_data)


# %%
# Account management events
def _extract_acct_mgmt_events(event_data):
    # Get a full list of Windows Security Events

    w_evt = pkgutil.get_data("msticpy", f"resources{os.sep}WinSecurityEvent.json")
    win_event_df = pd.read_json(w_evt)

    # Create criteria for events that we're interested in
    acct_sel = win_event_df["subcategory"] == "User Account Management"
    group_sel = win_event_df["subcategory"] == "Security Group Management"
    schtask_sel = (win_event_df["subcategory"] == "Other Object Access Events") & (
        win_event_df["description"].str.contains("scheduled task")
    )

    event_list = win_event_df[acct_sel | group_sel | schtask_sel]["event_id"].to_list()
    # Add Service install event
    event_list.append(7045)
    return event_data[event_data["EventID"].isin(event_list)]


def _create_acct_event_pivot(account_event_data):
    # Create a pivot of Event vs. Account
    win_events_acc = account_event_data[["Account", "Activity", "TimeGenerated"]].copy()
    win_events_acc = win_events_acc.replace("-\\-", "No Account").replace(
        {"Account": ""}, value="No Account"
    )
    win_events_acc["Account"] = win_events_acc.apply(
        lambda x: x.Account.split("\\")[-1], axis=1
    )
    event_pivot_df = (
        pd.pivot_table(
            win_events_acc,
            values="TimeGenerated",
            index=["Activity"],
            columns=["Account"],
            aggfunc="count",
        )
        .fillna(0)
        .reset_index()
    )
    return event_pivot_df


@set_text(
    title="Summary of Account Management Events on host",
    text="""
Yellow highlights indicate account with highest event count.
""",
)
def _display_acct_event_pivot(event_pivot_df):
    display(
        event_pivot_df.style.applymap(lambda x: "color: white" if x == 0 else "")
        .applymap(
            lambda x: "background-color: lightblue"
            if not isinstance(x, str) and x > 0
            else ""
        )
        .set_properties(subset=["Activity"], **{"width": "400px", "text-align": "left"})
        .highlight_max(axis=1)
        .hide_index()
    )


@set_text(title="Timeline of Account Management Events on host")
def _display_acct_mgmt_timeline(acct_event_data):
    # Plot events on a timeline
    return nbdisplay.display_timeline(
        data=acct_event_data,
        group_by="EventID",
        source_columns=["Activity", "Account"],
        legend="right",
    )


---
# More Info

## msticpy
- Documentation - https://msticpy.readthedocs.io
- GitHub - https://github.com/microsoft/msticpy
- PyPI - https://pypi.org/project/msticpy/

## msticnb - Notebooklets
- GitHub - https://github.com/microsoft/msticnb

## Notebooks
- Azure-Sentinel-Notebooks - https://github.com/Azure/Azure-Sentinel-Notebooks
- Binder-able demo - https://github.com/Azure/Azure-Sentinel-Notebooks/tree/master/nbdemo

--
## Network Flow Notebooklet

In [17]:
flow_summary = nb.nblts.azsent.network.NetworkFlowSummary()
flow_result = flow_summary.run(value="MSTICAlertsWin1", timespan=TimeSpan(time_selector=time_span))

Getting data from AzureNetworkAnalytics...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

,source,dest,L7Protocol,FlowDirection,TotalAllowedFlows
0,10.0.3.5,104.116.249.67,https,O,40
1,10.0.3.5,104.117.0.237,http,O,20
2,10.0.3.5,104.117.0.237,https,O,40
3,10.0.3.5,104.40.17.153,https,O,17
4,10.0.3.5,104.43.212.12,https,O,40
5,10.0.3.5,13.107.4.50,http,O,68
6,10.0.3.5,13.107.4.52,http,O,5
7,10.0.3.5,13.64.188.245,https,O,42
8,10.0.3.5,13.65.107.32,https,O,1191
9,10.0.3.5,13.67.143.117,https,O,309


Found 210 unique IP Addresses.
Getting data from Whois...
................................................................................................................................................................................................................

,DestASN,SourceASN,TotalAllowedFlows,L7Protocols,source_ips,dest_ips
0,"AKAMAI-AS, US",No ASN Information for IP type: Private,546.0,"[https, http]",[10.0.3.5],"[104.116.249.67, 104.117.0.237, 23.223.3.100, 23.32.68.208, 23.32.69.100, 23.4.187.27, 23.47.27...."
1,"AKAMAI-ASN1, EU",No ASN Information for IP type: Private,323.0,"[http, https]",[10.0.3.5],"[184.50.238.226, 184.51.150.105, 184.51.150.81, 23.215.130.137, 23.215.98.90, 23.219.93.42, 23.2..."
2,"AMAZON-02, US",No ASN Information for IP type: Private,37.0,"[https, http]",[10.0.3.5],"[99.84.104.63, 99.84.106.178, 99.84.106.27, 99.84.106.92]"
3,"BYTEMARK-AS, GB",No ASN Information for IP type: Private,5.0,[https],[10.0.3.5],[46.43.34.31]
4,"CENTURYLINK-US-LEGACY-QWEST, US",No ASN Information for IP type: Private,40.0,[http],[10.0.3.5],"[65.158.47.112, 65.158.47.34, 65.158.47.50, 67.135.105.19]"
5,"EDGECAST, US",No ASN Information for IP type: Private,1018.0,"[https, http]",[10.0.3.5],"[72.21.81.200, 72.21.81.240, 72.21.91.29]"
6,"GOOGLE, US",No ASN Information for IP type: Private,6359.0,"[https, http]",[10.0.3.5],"[172.217.10.35, 172.217.10.67, 172.217.11.10, 172.217.15.110, 172.217.15.67, 172.217.15.74, 172...."
7,"HIGHWINDS3, US",No ASN Information for IP type: Private,51.0,[http],[10.0.3.5],"[205.185.216.10, 205.185.216.42]"
8,"JUMP, GB",No ASN Information for IP type: Private,5.0,[https],[10.0.3.5],[212.13.197.231]
9,"LEVEL3, US",No ASN Information for IP type: Private,52.0,[http],[10.0.3.5],"[67.26.233.254, 67.26.237.254, 8.249.225.254, 8.249.231.254, 8.249.241.254, 8.253.139.120, 8.253..."


In [ ]:
flow_summary.select_asns()

In [ ]:
flow_summary.lookup_ti_for_asn_ips()
flow_summary.show_selected_asn_map()